<a href="https://colab.research.google.com/github/filchyboy/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/LS_DS_432_Convolution_Neural_Networks_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](./data/train/mountain/art1131.jpg)|![](./data/validation/forest/cdmc317.jpg)|

The problem is relatively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several different possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pre-trained layers from resnet
4. Report your model's accuracy

## Load in Data

This surprisingly more difficult than it seems, because you are working with directories of images instead of a single file. This boiler plate will help you download a zipped version of the directory of images. The directory is organized into "train" and "validation" which you can use inside an `ImageGenerator` class to stream batches of images thru your model.  


### Download & Summarize the Data

This step is completed for you. Just run the cells and review the results. 

In [0]:
import tensorflow as tf
import os

_URL = 'https://github.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data.zip?raw=true'

path_to_zip = tf.keras.utils.get_file('./data.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'data')

In [0]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [51]:
train_dir

'/root/.keras/datasets/./data/train'

In [0]:
train_mountain_dir = os.path.join(train_dir, 'mountain')  # directory with our training cat pictures
train_forest_dir = os.path.join(train_dir, 'forest')  # directory with our training dog pictures
validation_mountain_dir = os.path.join(validation_dir, 'mountain')  # directory with our validation cat pictures
validation_forest_dir = os.path.join(validation_dir, 'forest')  # directory with our validation dog pictures

In [0]:
num_mountain_tr = len(os.listdir(train_mountain_dir))
num_forest_tr = len(os.listdir(train_forest_dir))

num_mountain_val = len(os.listdir(validation_mountain_dir))
num_forest_val = len(os.listdir(validation_forest_dir))

total_train = num_mountain_tr + num_forest_tr
total_val = num_mountain_val + num_forest_val

In [54]:
print('total training mountain images:', num_mountain_tr)
print('total training forest images:', num_forest_tr)

print('total validation mountain images:', num_mountain_val)
print('total validation forest images:', num_forest_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

total training mountain images: 254
total training forest images: 270
total validation mountain images: 125
total validation forest images: 62
--
Total training images: 524
Total validation images: 187


### Keras `ImageGenerator` to Process the Data

This step is completed for you, but please review the code. The `ImageGenerator` class reads in batches of data from a directory and pass them to the model one batch at a time. Just like large text files, this method is advantageous, because it stifles the need to load a bunch of images into memory. 

Check out the documentation for this class method: [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class). You'll expand it's use in the third assignment objective.

In [0]:
batch_size = 16
epochs = 10
IMG_HEIGHT = 224
IMG_WIDTH = 224
CHNLS = 3

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [57]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 533 images belonging to 2 classes.


In [58]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 195 images belonging to 2 classes.


In [59]:
type(val_data_gen)

keras_preprocessing.image.directory_iterator.DirectoryIterator

In [60]:
val_data_gen[0]

(array([[[[0.01176471, 0.41960788, 0.7607844 ],
          [0.01568628, 0.42352945, 0.76470596],
          [0.01568628, 0.42352945, 0.76470596],
          ...,
          [0.06666667, 0.47058827, 0.8313726 ],
          [0.06666667, 0.47058827, 0.8313726 ],
          [0.07450981, 0.4666667 , 0.8352942 ]],
 
         [[0.01960784, 0.427451  , 0.7686275 ],
          [0.01568628, 0.42352945, 0.76470596],
          [0.01568628, 0.42352945, 0.76470596],
          ...,
          [0.07450981, 0.4784314 , 0.83921576],
          [0.08235294, 0.47450984, 0.8431373 ],
          [0.08235294, 0.47450984, 0.8431373 ]],
 
         [[0.01568628, 0.43137258, 0.77647066],
          [0.01568628, 0.43137258, 0.77647066],
          [0.01568628, 0.43137258, 0.77647066],
          ...,
          [0.09019608, 0.48235297, 0.85098046],
          [0.09019608, 0.48235297, 0.85098046],
          [0.09019608, 0.48235297, 0.85098046]],
 
         ...,
 
         [[0.45098042, 0.25490198, 0.02352941],
          [0.33725

## Instatiate Model

In [0]:
import numpy as np
 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API
 
resnet = ResNet50(weights='imagenet', include_top=False)

In [0]:
for layer in resnet.layers:
    layer.trainable = False

In [63]:
type(resnet)

tensorflow.python.keras.engine.training.Model

In [0]:
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Fit Model

In [66]:
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/10
32/32 [==============================] - 3s 101ms/step - loss: 0.8963 - accuracy: 0.5176 - val_loss: 0.9001 - val_accuracy: 0.3466
Epoch 2/10
32/32 [==============================] - 2s 76ms/step - loss: 0.6241 - accuracy: 0.6327 - val_loss: 0.5511 - val_accuracy: 0.7914
Epoch 3/10
32/32 [==============================] - 2s 77ms/step - loss: 0.5211 - accuracy: 0.7944 - val_loss: 0.5857 - val_accuracy: 0.7730
Epoch 4/10
32/32 [==============================] - 2s 77ms/step - loss: 0.5052 - accuracy: 0.7625 - val_loss: 0.4820 - val_accuracy: 0.8160
Epoch 5/10
32/32 [==============================] - 2s 77ms/step - loss: 0.4282 - accuracy: 0.8503 - val_loss: 0.4091 - val_accuracy: 0.8896
Epoch 6/10
32/32 [==============================] - 2s 77ms/step - loss: 0.4290 - accuracy: 0.8044 - val_loss: 0.5630 - val_accuracy: 0.7239
Epoch 7/10
32/32 [==============================] - 2s 78ms/step - loss: 0.3639 - accuracy: 0.8802 - val_loss: 0.3242 - val_accuracy: 0.9264
Epoch 8/10
3

In [69]:
val_data_gen[0]

(array([[[[0.03529412, 0.04313726, 0.02352941],
          [0.        , 0.01960784, 0.        ],
          [0.03921569, 0.05882353, 0.04313726],
          ...,
          [0.04705883, 0.03921569, 0.04313726],
          [0.04313726, 0.03529412, 0.03921569],
          [0.02745098, 0.01960784, 0.02352941]],
 
         [[0.        , 0.00784314, 0.        ],
          [0.        , 0.01568628, 0.        ],
          [0.        , 0.01568628, 0.        ],
          ...,
          [0.04705883, 0.03921569, 0.04313726],
          [0.04313726, 0.03529412, 0.03921569],
          [0.03529412, 0.02745098, 0.03137255]],
 
         [[0.0627451 , 0.08627451, 0.04705883],
          [0.12941177, 0.15294118, 0.1137255 ],
          [0.10588236, 0.13725491, 0.09411766],
          ...,
          [0.        , 0.        , 0.        ],
          [0.01960784, 0.01960784, 0.01960784],
          [0.0627451 , 0.0627451 , 0.0627451 ]],
 
         ...,
 
         [[0.05882353, 0.07450981, 0.08627451],
          [0.02745

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [36]:
# Define the Model
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

Using TensorFlow backend.


In [37]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 1s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [0]:
# Compile Model

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [45]:
# Fit Model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 5s 82us/step - loss: 0.0065 - accuracy: 0.9980 - val_loss: 0.0038 - val_accuracy: 0.9985
Epoch 2/12
60000/60000 [==============================] - 5s 76us/step - loss: 0.0063 - accuracy: 0.9981 - val_loss: 0.0042 - val_accuracy: 0.9986
Epoch 3/12
60000/60000 [==============================] - 5s 76us/step - loss: 0.0055 - accuracy: 0.9982 - val_loss: 0.0043 - val_accuracy: 0.9986
Epoch 4/12
60000/60000 [==============================] - 5s 76us/step - loss: 0.0055 - accuracy: 0.9983 - val_loss: 0.0036 - val_accuracy: 0.9988
Epoch 5/12
60000/60000 [==============================] - 5s 77us/step - loss: 0.0054 - accuracy: 0.9983 - val_loss: 0.0035 - val_accuracy: 0.9987
Epoch 6/12
60000/60000 [==============================] - 5s 77us/step - loss: 0.0049 - accuracy: 0.9984 - val_loss: 0.0041 - val_accuracy: 0.9988
Epoch 7/12
60000/60000 [==============================] - 5s 76us/st

# Custom CNN Model with Image Manipulations

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Simply, you should be able to modify our image generator for the problem. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [0]:
# State Code for Image Manipulation Here

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language